In [3]:
import pandas as pd
import numpy as np
import sklearn as sk
import gc
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings('ignore')


#Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

bureau = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau.csv')




In [4]:
#La imputación de valores nulos y la normalización se hará cuando lo juntemos con la tabla principal.
#Por el momento vamos a hacer el feature engineering de esta tabla y la codificación de variables categóricas.

#SK_ID_BUREAU nos indica la cantidad de créditos que ha tenido un cliente. Vamos a juntarlo a agruparlo con SK_ID_CURR
#para tener una variable que nos indique la cantidad de créditos que ha tenido un cliente.

count_bureau_loans = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby(['SK_ID_CURR'], as_index=False).count()
count_bureau_loans.rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT'}, inplace = True)

In [5]:
#Vamos a crear algunas variables 

#En relación a los días
bureau['DAYS_REMAINING'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_CREDIT']
bureau['DAYS_UPDATE_CREDIT'] = bureau['DAYS_CREDIT_UPDATE'] - bureau['DAYS_CREDIT']

#En relación a los Overdues
bureau['OVERDUE']=bureau['CREDIT_DAY_OVERDUE'].map(lambda x: 1 if x>0 else 0)
bureau['DAYS_OVERDUE_30_DAYS_RATIO'] = bureau['AMT_CREDIT_MAX_OVERDUE'] / 30
bureau['DAYS_OVERDUE_90_DAYS_RATIO'] = bureau['AMT_CREDIT_MAX_OVERDUE'] / 90
bureau['DAYS_OVERDUE_180_DAYS_RATIO'] = bureau['AMT_CREDIT_MAX_OVERDUE'] / 180

#En relación a los prolongados
bureau['DAYS_OVERDUE_TO_PROLONGED_RATIO'] = bureau['AMT_CREDIT_MAX_OVERDUE'] / bureau['CNT_CREDIT_PROLONG']
bureau['AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO'] = bureau['AMT_CREDIT_SUM_OVERDUE'] / bureau['CNT_CREDIT_PROLONG']
bureau['AMT_CREDIT_MAX_OVERDUE_TO_PROLONGED_RATIO'] = bureau['AMT_CREDIT_MAX_OVERDUE'] / bureau['CNT_CREDIT_PROLONG']

#En relación a los "montos"
bureau['AMT_CREDIT_SUM_DEBT_TO_AMT_CREDIT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']
bureau['AMT_CREDIT_SUM_DEBT_TO_AMT_CREDIT_SUM_DIFF'] = bureau['AMT_CREDIT_SUM_DEBT'] - bureau['AMT_CREDIT_SUM']
bureau['AMT_CREDIT_SUM_TO_AMT_CREDIT_SUM_OVERDUE_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_OVERDUE']
bureau['AMT_CREDIT_SUM_TO_AMT_CREDIT_SUM_OVERDUE_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_OVERDUE']
bureau['AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM_LIMIT'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_DIFF'] = bureau['AMT_CREDIT_SUM_LIMIT'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO'] = bureau['AMT_CREDIT_SUM_LIMIT'] / bureau['AMT_CREDIT_SUM_OVERDUE']
bureau['AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_DIFF'] = bureau['AMT_CREDIT_SUM_LIMIT'] - bureau['AMT_CREDIT_SUM_OVERDUE']
bureau['AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_LIMIT'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_DEBT_DIFF'] = bureau['AMT_CREDIT_SUM_LIMIT'] - bureau['AMT_CREDIT_SUM_DEBT']

In [6]:
#Agregaciones
bureau_numerical_agg = {
    #Variables base
    'DAYS_CREDIT': ['mean', 'max', 'min', 'sum'],
    'CREDIT_DAY_OVERDUE': ['mean', 'max', 'min'],
    'DAYS_CREDIT_ENDDATE': ['mean', 'max', 'min'],
    'DAYS_ENDDATE_FACT': ['mean', 'max', 'min'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max', 'min', 'sum'],
    'CNT_CREDIT_PROLONG': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM': ['mean', 'max', 'min', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max', 'min', 'sum'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'max', 'min', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max', 'min', 'sum'],
    'DAYS_CREDIT_UPDATE': ['mean', 'max', 'min'],
    'DAYS_REMAINING': ['mean', 'max', 'min'],
    'DAYS_UPDATE_CREDIT': ['mean', 'max', 'min'],

    #En base a algunas variables creadas
    #En relación a los días
    'DAYS_REMAINING': ['mean', 'max', 'min'],
    'DAYS_UPDATE_CREDIT': ['mean', 'max', 'min'],
    'DAYS_OVERDUE_30_DAYS_RATIO': ['mean', 'max', 'min'],
    'DAYS_OVERDUE_90_DAYS_RATIO': ['mean', 'max', 'min'],
    'DAYS_OVERDUE_180_DAYS_RATIO': ['mean', 'max', 'min'],
    #En relación a las prolongaciones
    'DAYS_OVERDUE_TO_PROLONGED_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_MAX_OVERDUE_TO_PROLONGED_RATIO': ['mean', 'max', 'min'],

    #En relación a los montos
    'AMT_CREDIT_SUM_DEBT_TO_AMT_CREDIT_SUM_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_DEBT_TO_AMT_CREDIT_SUM_DIFF': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_TO_AMT_CREDIT_SUM_OVERDUE_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_TO_AMT_CREDIT_SUM_OVERDUE_DIFF': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_DIFF': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_DIFF': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_DEBT_RATIO': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_DEBT_DIFF': ['mean', 'max', 'min'],
}

bureau_agg = bureau.groupby('SK_ID_CURR').agg({**bureau_numerical_agg})
bureau_agg.columns = [
    f"{col[0]}_{col[1].upper()}_(BUREAU)" for col in bureau_agg.columns.ravel()
]

In [7]:
temp_ratio = (
    bureau
    .groupby('SK_ID_CURR')
    .agg({'AMT_CREDIT_SUM_DEBT':'sum', 'AMT_CREDIT_SUM':'sum'})
    .rename(columns={
        'AMT_CREDIT_SUM_DEBT':'sum_debt',
        'AMT_CREDIT_SUM':'sum_credit'
    })
)
temp_ratio['debt_credit_ratio_None'] = temp_ratio['sum_debt'] / temp_ratio['sum_credit']

temp_ratio = temp_ratio[['debt_credit_ratio_None']]  

# Merge con bureau_agg o con tu DF principal
bureau_agg = bureau_agg.merge(temp_ratio, on='SK_ID_CURR', how='left')

In [8]:
# Filtrar active
bureau_active = bureau[bureau['CREDIT_ACTIVE']=='Active']
bureau_active_agg = bureau_active.groupby('SK_ID_CURR').agg(bureau_numerical_agg)

# Quitar DAYS_ENDDATE_FACT si solo aplica a cerrados
if 'DAYS_ENDDATE_FACT' in bureau_active_agg.columns:
    bureau_active_agg.drop('DAYS_ENDDATE_FACT', axis=1, inplace=True, level=0)
    # (level=0 si es MultiIndex)

bureau_active_agg.columns = [
    f"ACTIVE_{col[0]}_{col[1].upper()}_(BUREAU)" for col in bureau_active_agg.columns.ravel()
]

# Filtrar closed
bureau_closed = bureau[bureau['CREDIT_ACTIVE']=='Closed']
bureau_closed_agg = bureau_closed.groupby('SK_ID_CURR').agg(bureau_numerical_agg)
bureau_closed_agg.columns = [
    f"CLOSED_{col[0]}_{col[1].upper()}_(BUREAU)" for col in bureau_closed_agg.columns.ravel()
]

# Uno las agregaciones de créditos activos
bureau_agg = bureau_agg.merge(bureau_active_agg, on='SK_ID_CURR', how='left')

# Uno las agregaciones de créditos cerrados
bureau_agg = bureau_agg.merge(bureau_closed_agg, on='SK_ID_CURR', how='left')

columns_active = bureau_active_agg.columns  # Son las col de agregaciones de active
# columns_active es algo como: Index(['ACTIVE_DAYS_CREDIT_MEAN_(BUREAU)', 'ACTIVE_DAYS_CREDIT_MAX_(BUREAU)', ...])

for col_name in columns_active:
    # col_name = 'ACTIVE_DAYS_CREDIT_MEAN_(BUREAU)'
    # la versión 'CLOSED' se obtiene sustituyendo 'ACTIVE_' por 'CLOSED_'
    closed_col_name = col_name.replace('ACTIVE_', 'CLOSED_')
    
    ratio_col_name = col_name.replace('ACTIVE_', 'RATIO_ACTIVE_TO_CLOSED_')
    # ratio_col_name algo como: 'RATIO_ACTIVE_TO_CLOSED_DAYS_CREDIT_MEAN_(BUREAU)'

    if closed_col_name in bureau_agg.columns:
        bureau_agg[ratio_col_name] = bureau_agg[col_name] / bureau_agg[closed_col_name]
    else:
        # Por si no existe en closed
        bureau_agg[ratio_col_name] = None

In [9]:
last_active_df = (
    bureau[bureau['CREDIT_ACTIVE']=='Active']
    .groupby('SK_ID_CURR')['DAYS_CREDIT']
    .max()  # max => más cercano a 0 => más reciente
    .reset_index()
    .rename(columns={'DAYS_CREDIT': 'last_active_DAYS_CREDIT'})
)

bureau_agg = bureau_agg.merge(last_active_df, on='SK_ID_CURR', how='left')

In [ ]:
#Incorporamos todos los cambios realizados a nuestro dataframe principal bureau
"""Y SI EN VEZ DE HACERLO TODO SEPARADO, PASO AQUÍ EL APPLICATION_TRAIN_DEF Y VOY HACIENDO LOS MERGES AQUI Y AL FINAL LO EXPORTO TODO?? EN VEZ DE PASARSELO
A LA TABLA PRINCIPAL???"""

bureau = bureau.merge(bureau_agg, on='SK_ID_CURR', how='left')
print(bureau.info())

In [11]:
#Vamos a tratar con las variables categóricas
variables_categoricas_bureau = bureau.select_dtypes('object').columns
